In [ ]:
import pandas as pd
from pathlib import Path


# gettting the range of the intensity and the number of pepetides
import pandas as pd
import os
from pathlib import Path
import numpy as np
direcroy_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/2023.06.22_AhS_PAPER_COHORT'
transcripts_path = "/media/kusterlab/internal_projects/active/TOPAS/WP31/FPKM/fpkms_230725mitINFORM.csv"
basket_annotation_path = "/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_MTBs_Evaluation/TUPAC_SCORING_4th gen_230628.xlsx"
patietns_meta_data = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_MTBs_Evaluation/Metadata_Papercohort_230801.xlsx'





# intensity files
PREPROCESSED_FP_INTENSITY = 'preprocessed_fp.csv'

REGEX = r'(^\S+-.+-\S)|(Reporter intensity corrected)|(^P\d{6}$)'
Z_SCORES_PATH = 'full_proteome_measures_z.tsv'
Z_SCORE_REGEX = r'(^\S+-.+-\S)|(Reporter intensity corrected)|(^P\d{6}$) Z-score'
BASKET_SCORES_PATH = 'basket_scores_4th_gen.tsv'

REGEX_META = r'^Identification metadata'  # to get num identified pepetides in FP intensity file
def replace_statement(x):
    x = x.replace('num_peptides=','')
    x = x.replace('|','')
    x = x.replace(';','')
    x = x.replace('detected in batch','0')
    return x


def unnest_proteingroups(df:pd.DataFrame) -> pd.DataFrame:
    """
    Unnest the protein_groups A;B as two separate rows with the same values
    the protein groups are the index of the the pandas dataframe df
    """
    temp_df = df
    temp_df['index'] = temp_df.index.str.split(';')
    temp_df = temp_df.explode('index')
    temp_df = temp_df.set_index('index')
    return temp_df


def get_corelation_scatter_plot_per_prtein(protein:str,transcripts_z_scores_df,proteomics_df,plot_saving_dir):
    import seaborn as sns
    import matplotlib.pyplot as plt
    df1 = pd.DataFrame(transcripts_z_scores_df.loc[protein,:])
    df2 = pd.DataFrame(proteomics_df.loc[protein,:])
    df1.columns = ['transcripts']
    df2.columns = ['proteomics']
    all = pd.concat([df2,df1],axis=1)
    sns.scatterplot(data=all, x="transcripts", y="proteomics")
    plt.ylim(-4, 4) 
    plt.xlim(-4, 4)
    plt.savefig(f'{plot_saving_dir}/{protein}.svg',dpi=300)


def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))


def setdiff(lst1,lst2):
    return(list(set(lst1) - set(lst2)))


def get_correlation(protein:str,df1,df2):
    ans = None
    try:
        df1 = pd.DataFrame(df1.loc[protein,:])
        df2 = pd.DataFrame(df2.loc[protein,:])
        all_df= pd.concat([df1,df2],axis=1)
        all_df = all_df.dropna()
        if len(all_df) > 20:
            ans = all_df.corr().iloc[0,1]
    except:
        print(f'{protein} lead to error we put NA')
        pass
    return ans,len(all_df)



def get_pep_number_from_protein_name(intensity_df:pd.DataFrame,protein_name:str,regex= r'^Identification metadata') -> pd.DataFrame:
    """ 
    gets the number of the identified pepetides from the protein name across all the patients
    :intensity_df: A pandas dataframe of the intensities with Identification metadata columns for each patient 
    :protein_name: the name of the protein
    :USAGE :
        get_pep_number_from_protein_name(fp_df,'EGFR')

    """
    try:
        #print(protein_name)
        premeta_df = pd.DataFrame(intensity_df.loc[protein_name,:])
        premeta_df = premeta_df.filter(regex=regex,axis=0)
        premeta_df[protein_name] = premeta_df[protein_name].str.replace('num_peptides=|;','')
        premeta_df[protein_name] = premeta_df[protein_name].str.replace('detected in batch','0') 
        premeta_df[protein_name] = pd.to_numeric(premeta_df[protein_name])
        premeta_df['Sample name'] = premeta_df.index.str.replace('Identification metadata ','')
        premeta_df.columns = ['num_pep','Sample name']
        return premeta_df
    except:
        pass



def load_intensity_meta_data(instensitypath,key,regex= r'^Identification metadata' ):
    print(f'reading {instensitypath}')
    if os.path.exists(instensitypath):
        cols = pd.read_csv(instensitypath,low_memory=False, nrows=10).filter(regex=regex).columns.tolist()
        cols.append(key)
        intensity_df = pd.read_csv(instensitypath,low_memory=False, usecols=cols)
        intensity_df.index = intensity_df[key]
        intensity_df = intensity_df.loc[:,~intensity_df.columns.duplicated()]
        # intensity_df = intensity_df.fillna(0)
        print('Intensity meta_data loaded')
        return intensity_df.fillna('num_peptides=0;')
    else:
        return [f'{instensitypath} did not exist']



def get_cv_from_melted_df(melted_df:pd.DataFrame,protein_col='Proteins_names',value_col='value') -> pd.DataFrame:
    """
    calculated CV in a long format dataFrame
    :melted_df: long_format data_Frame with at least two columns one for the protein names and one for the values
    :protein_col: the column name of the proteins
    :value_col: the column of the values
    """
    cv_df = melted_df.groupby(protein_col)[value_col].agg(['std','mean'])
    cv_df['cv'] = cv_df['std']/cv_df['mean']
    cv_df['Gene names'] = cv_df.index
    return cv_df



def protein_remover_by_sparcity(df:pd.DataFrame,minimum_samples_inside=20) -> pd.DataFrame:
    """
    Removes the sparcity based on the defined level from the dataframe
    :df: dataframe with the proteins as the index and the samples as the columns
    :minimum_samples_inside: the minimun number of samples without NA values that should exist not to exclude that protein
    
    """
    df = df.copy()
    nan_count = len(df.columns) - df.isna().sum(axis = 1)
    df.drop(index= nan_count[nan_count<minimum_samples_inside].index,inplace=True)
    return df



